In [43]:
import open3d as o3d
import open3d.visualization as vi
import pickle
import numpy as np
from plyfile import PlyData

def get_camera_frustum(img_size, K, W2C, frustum_length=0.5, color=[0., 1., 0.]):
    W, H = img_size
    hfov = np.rad2deg(np.arctan(W / 2. / K[0, 0]) * 2.)
    vfov = np.rad2deg(np.arctan(H / 2. / K[1, 1]) * 2.)
    half_w = 0.00005 * np.tan(np.deg2rad(hfov / 2.))
    half_h = 0.00005 * np.tan(np.deg2rad(vfov / 2.))

    # build view frustum for camera (I, 0)
    frustum_points = np.array([[0., 0., 0.],                          # frustum origin
                               [-half_w, -half_h, frustum_length],    # top-left image corner
                               [half_w, -half_h, frustum_length],     # top-right image corner
                               [half_w, half_h, frustum_length],      # bottom-right image corner
                               [-half_w, half_h, frustum_length]])    # bottom-left image corner
    frustum_lines = np.array([[0, i] for i in range(1, 5)] + [[i, (i+1)] for i in range(1, 4)] + [[4, 1]])
    frustum_colors = np.tile(np.array(color).reshape((1, 3)), (frustum_lines.shape[0], 1))

    # frustum_colors = np.vstack((np.tile(np.array([[1., 0., 0.]]), (4, 1)),
    #                            np.tile(np.array([[0., 1., 0.]]), (4, 1))))

    # transform view frustum from (I, 0) to (R, t)
    C2W = np.linalg.inv(W2C)
    frustum_points = np.dot(np.hstack((frustum_points, np.ones_like(frustum_points[:, 0:1]))), C2W.T)
    frustum_points = frustum_points[:, :3] / frustum_points[:, 3:4]

    return frustum_points, frustum_lines, frustum_colors


def frustums2lineset(frustums):
    N = len(frustums)
    merged_points = np.zeros((N*5, 3))      # 5 vertices per frustum
    merged_lines = np.zeros((N*8, 2))       # 8 lines per frustum
    merged_colors = np.zeros((N*8, 3))      # each line gets a color

    for i, (frustum_points, frustum_lines, frustum_colors) in enumerate(frustums):
        merged_points[i*5:(i+1)*5, :] = frustum_points
        merged_lines[i*8:(i+1)*8, :] = frustum_lines + i*5
        merged_colors[i*8:(i+1)*8, :] = frustum_colors

    lineset = o3d.geometry.LineSet()
    lineset.points = o3d.utility.Vector3dVector(merged_points)
    lineset.lines = o3d.utility.Vector2iVector(merged_lines)
    lineset.colors = o3d.utility.Vector3dVector(merged_colors)

    return lineset

def frustums2mesh(frustums):
    N = len(frustums)
    merged_vertices = np.zeros((N*5, 3))      # 5 vertices per frustum
    merged_triangles = np.zeros((N*2, 3), dtype=np.int32)  # 2 triangles per frustum

    for i, (frustum_points, _, _) in enumerate(frustums):
        merged_vertices[i*5:(i+1)*5, :] = frustum_points
        # Create two triangles to form the image plane
        merged_triangles[i*2, :] = [i*5+1, i*5+2, i*5+3]
        merged_triangles[i*2+1, :] = [i*5+1, i*5+3, i*5+4]

    mesh = vi.rendering.MaterialRecord()
    mesh.shader = 'defaultLitSSR'
    mesh.baseColor = [1.0, 0.0, 0.0, 0.5]
    mesh.vertices = o3d.utility.Vector3dVector(merged_vertices)
    mesh.triangles = o3d.utility.Vector3iVector(merged_triangles)
    # Color the image plane red

    return mesh

def visualize_cameras(colored_camera_dicts, sphere_radius, camera_size=0.1, geometry_file=None, geometry_type='mesh'):
    # sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius, resolution=10)
    # sphere = o3d.geometry.LineSet.create_from_triangle_mesh(sphere)
    # sphere.paint_uniform_color((0, 0, 0))

    coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0., 0., 0.])
    things_to_draw = [coord_frame]

    idx = 0
    for color, camera_dict in colored_camera_dicts:
        idx += 1

        cnt = 0
        frustums = []
        for i in range(len(camera_dict)):
            # [[R, T, FovY, FovX, width, height], ...]
            K = np.array([[camera_dict[i][3], 0, camera_dict[i][4]/2, 0],
                          [0, camera_dict[i][2], camera_dict[i][5]/2, 0],
                          [0, 0, 1, 0],
                          [0, 0, 0, 1]])

            # generate W2C matrix from R and T
            W2C = np.eye(4)
            W2C[:3, :3] = camera_dict[i][0].transpose()
            W2C[:3, 3] = camera_dict[i][1]
            C2W = np.linalg.inv(W2C)

            img_size = (camera_dict[i][4], camera_dict[i][5])
            frustums.append(get_camera_frustum(img_size, K, W2C, frustum_length=camera_size, color=color))
            cnt += 1

        # for img_name in sorted(camera_dict.keys()):
        #     K = np.array(camera_dict[img_name]['K']).reshape((4, 4))
        #     W2C = np.array(camera_dict[img_name]['W2C']).reshape((4, 4))
        #     C2W = np.linalg.inv(W2C)
        #     img_size = camera_dict[img_name]['img_size']
        #     frustums.append(get_camera_frustum(img_size, K, W2C, frustum_length=camera_size, color=color))
        #     cnt += 1
        cameras = frustums2lineset(frustums)
        # cameras = frustums2mesh(frustums)
        things_to_draw.append(cameras)

    if geometry_file is not None:
        if geometry_type == 'mesh':
            geometry = o3d.io.read_triangle_mesh(geometry_file)
            geometry.compute_vertex_normals()
        elif geometry_type == 'pointcloud':
            geometry = o3d.io.read_point_cloud(geometry_file)
            # add colors
            plydata = PlyData.read(geometry_file)
            xyz = np.stack((np.asarray(plydata.elements[0]["x"]),
                        np.asarray(plydata.elements[0]["y"]),
                        np.asarray(plydata.elements[0]["z"])),  axis=1)
            features_dc = np.zeros((xyz.shape[0], 3, 1))
            print(plydata.elements[0])
            features_dc[:, 0, 0] = np.asarray(plydata.elements[0]["red"])
            features_dc[:, 1, 0] = np.asarray(plydata.elements[0]["green"])
            features_dc[:, 2, 0] = np.asarray(plydata.elements[0]["blue"])
            print(features_dc.shape)
            print(np.asarray(geometry.colors))
            ## draw color to pcd
            geometry.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

            # geometry.paint_uniform_color([1.0, 0.0, 0.0])
            print(np.asarray(geometry.colors))
        

        else:
            raise Exception('Unknown geometry_type: ', geometry_type)

        things_to_draw.append(geometry)

    # o3d.visualization.draw_geometries(things_to_draw)
    # Create a Visualizer object
    vis = o3d.visualization.Visualizer()

    # Add the geometry to the Visualizer
    vis.create_window()
    for thing in things_to_draw:
        vis.add_geometry(thing)

    # Change the visualization mode
    render_option = vis.get_render_option()
    render_option.point_size = 1.0  # Change the point size
    # render_option.background_color = np.array([0, 0, 0])  # Change the background color

    # Draw the geometries
    vis.run()


    
def cam_vis(gt_cam, perturb_cam, scale=1.0):
    '''
    Visualize the camera position and orientation
    Blue color is for gt camera and red color is for perturbed camera.

    gt_cam[0]: CameraInfo(uid=1, R=array([[ 0.25976008,  0.24620705,  0.93375949],
       [ 0.02419547,  0.96498892, -0.26117228],
       [-0.96537002,  0.09043488,  0.24470851]]), T=array([-1.47549721e-01, -2.61911888e-03,  4.14421572e+00]), FovY=0.6823680471232931, FovX=0.9775767040029059, image=<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4946x3286 at 0x7F6EF0923370>, image_path='/home/cvlab05/project/sda/colmap/bicycle/images/_DSC8680.JPG', image_name='_DSC8680', width=4946, height=3286)
    '''

    colored_camera_dicts = [([0, 0, 1], gt_cam),
                            ([1, 0, 0], perturb_cam)
                            ]
    
    camera_size = 0.3

    # 4x4 camera intrinsics matrix
    # K = np.array([[gt_cam[0].FovX, 0, gt_cam[0].width/2, 0],
    #               [0, gt_cam[0].FovY, gt_cam[0].height/2, 0],
    #               [0, 0, 1, 0],
    #               [0, 0, 0, 1]])
    
    visualize_cameras(colored_camera_dicts, sphere_radius=1., camera_size=camera_size, geometry_file='/home/onground/projects/3dgs/bicycle_cam/points3D.ply', geometry_type='pointcloud')
        
if __name__ == '__main__':
    gt_cam = pickle.load(open('bicycle_pose_debug_barf/cam_gt.pkl', 'rb'))
    perturb_cam = pickle.load(open('bicycle_pose_debug_barf/cam_perturb.pkl', 'rb'))
    # [[R, T, FovY, FovX, width, height], ...]

    cam_vis(gt_cam, perturb_cam)


element vertex 56109
property float x
property float y
property float z
property float nx
property float ny
property float nz
property uchar red
property uchar green
property uchar blue
(56109, 3, 1)
[[0.48235294 0.49019608 0.47058824]
 [0.99607843 0.99607843 0.98823529]
 [0.98823529 0.98823529 0.99607843]
 ...
 [0.19607843 0.21960784 0.16862745]
 [0.30196078 0.30588235 0.18823529]
 [0.66666667 0.67058824 0.58039216]]
[[0.48235294 0.49019608 0.47058824]
 [0.99607843 0.99607843 0.98823529]
 [0.98823529 0.98823529 0.99607843]
 ...
 [0.19607843 0.21960784 0.16862745]
 [0.30196078 0.30588235 0.18823529]
 [0.66666667 0.67058824 0.58039216]]
